In [6]:
print(tf.__version__)

2.0.0-beta1


In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from tensorflow.python.ops import lookup_ops
from tensorflow.python.training.tracking import tracking


from absl import app
from absl import flags

import tensorflow.compat.v2 as tf
import os
import tempfile


In [2]:
from tensorflow.keras.layers import Dense, Flatten, Embedding, LSTM,Input,Embedding
from tensorflow.keras import Model
from tensorflow.keras.layers import BatchNormalization, Dropout,GlobalMaxPooling1D,GlobalAveragePooling1D,concatenate


In [3]:
#@tf.function(input_signature=[tf.TensorSpec([None], tf.dtypes.int64)])
class LanguageModelEncoder(Model):
    def __init__(self,vocab_size,emb_dim,state_size,n_layers):
        super(LanguageModelEncoder, self).__init__()
        self._state_size = state_size
        self.embedding_layer = Embedding(vocab_size,emb_dim)
        self._lstm_layers = [LSTM(self._state_size,return_sequences=True) for i in range(n_layers)]
        #self._lstm_layer = tf.keras.layers.LSTM(state_size,return_sequences=True)
        
    def __call__(self,sentence_lookup_ids):
        
        emb_output = self.embedding_layer(sentence_lookup_ids)
        lstm_output = emb_output # initialize to the input
        for lstm_layer in self._lstm_layers:
            lstm_output = lstm_layer(lstm_output)
        return lstm_output
        
        

In [4]:
def write_vocabulary_file(vocabulary):
  """Write temporary vocab file for module construction."""
  tmpdir = tempfile.mkdtemp()
  vocabulary_file = os.path.join(tmpdir, "tokens.txt")
  with tf.io.gfile.GFile(vocabulary_file, "w") as f:
    for entry in vocabulary:
      f.write(entry + "\n")
  return vocabulary_file

In [7]:
class ULMFiTModule(tf.train.Checkpoint):
  """
  LATER 
  """

  def __init__(self, vocab, emb_dim, buckets, state_size,n_layers):
    super(ULMFiTModule, self).__init__()
    self._buckets = buckets
    self._vocab_size = len(vocab)
    self.emb_row_size = self._vocab_size+self._buckets
    #self._embeddings = tf.Variable(tf.random.uniform(shape=[self.emb_row_size, emb_dim]))
    self._state_size = state_size
    self.model = LanguageModelEncoder(self.emb_row_size,emb_dim,state_size,n_layers)
    self._vocabulary_file = tracking.TrackableAsset(write_vocabulary_file(vocab)) 
    self.w2i_table = lookup_ops.index_table_from_file(
                    vocabulary_file= self._vocabulary_file,
                    num_oov_buckets=self._buckets,
                    hasher_spec=lookup_ops.FastHashSpec)
    self.i2w_table = lookup_ops.index_to_string_table_from_file(
                    vocabulary_file=self._vocabulary_file, 
                    delimiter = '\n',
                    default_value="UNKNOWN")
    self._logit_layer = tf.keras.layers.Dense(self.emb_row_size)


    
  def _tokenize(self, sentences):
    # Perform a minimalistic text preprocessing by removing punctuation and
    # splitting on spaces.
    normalized_sentences = tf.strings.regex_replace(
        input=sentences, pattern=r"\pP", rewrite="")
    sparse_tokens = tf.strings.split(normalized_sentences, " ").to_sparse()

    # Deal with a corner case: there is one empty sentence.
    sparse_tokens, _ = tf.sparse.fill_empty_rows(sparse_tokens, tf.constant(""))
    # Deal with a corner case: all sentences are empty.
    sparse_tokens = tf.sparse.reset_shape(sparse_tokens)

    return (sparse_tokens.indices, sparse_tokens.values,
            sparse_tokens.dense_shape)
    
  def _indices_to_words(self, indices):
    #return tf.gather(self._vocab_tensor, indices)
    return self.i2w_table.lookup(indices)
    

  def _words_to_indices(self, words):
    #return tf.strings.to_hash_bucket(words, self._buckets)
    return self.w2i_table.lookup(words)
  
  @tf.function(input_signature=[tf.TensorSpec([None],tf.dtypes.string)])   
  def _tokens_to_lookup_ids(self,sentences):
    token_ids, token_values, token_dense_shape = self._tokenize(sentences)
    tokens_sparse = tf.sparse.SparseTensor(
        indices=token_ids, values=token_values, dense_shape=token_dense_shape)
    tokens = tf.sparse.to_dense(tokens_sparse, default_value="")

    sparse_lookup_ids = tf.sparse.SparseTensor(
        indices=tokens_sparse.indices,
        values=self._words_to_indices(tokens_sparse.values),
        dense_shape=tokens_sparse.dense_shape)
    lookup_ids = tf.sparse.to_dense(sparse_lookup_ids, default_value=0)
    return tokens,lookup_ids
        
    

  @tf.function(input_signature=[tf.TensorSpec([None], tf.dtypes.string)])
  def train(self, sentences):
    tokens,lookup_ids = self._tokens_to_lookup_ids(sentences)
    # Targets are the next word for each word of the sentence.
    tokens_ids_seq = lookup_ids[:, 0:-1]
    tokens_ids_target = lookup_ids[:, 1:]
    tokens_prefix = tokens[:, 0:-1]

    # Mask determining which positions we care about for a loss: all positions
    # that have a valid non-terminal token.
    mask = tf.logical_and(
        tf.logical_not(tf.equal(tokens_prefix, "")),
        tf.logical_not(tf.equal(tokens_prefix, "<E>")))

    input_mask = tf.cast(mask, tf.int32)

    with tf.GradientTape() as t:
      #sentence_embeddings = tf.nn.embedding_lookup(self._embeddings,tokens_ids_seq)
    
      lstm_output = self.model(tokens_ids_seq)
      lstm_output = tf.reshape(lstm_output, [-1,self._state_size])
      logits = self._logit_layer(lstm_output)
      

      targets = tf.reshape(tokens_ids_target, [-1])
      weights = tf.cast(tf.reshape(input_mask, [-1]), tf.float32)

      losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
          labels=targets, logits=logits)

      # Final loss is the mean loss for all token losses.
      final_loss = tf.math.divide(
          tf.reduce_sum(tf.multiply(losses, weights)),
          tf.reduce_sum(weights),
          name="final_loss")

    watched = t.watched_variables()
    gradients = t.gradient(final_loss, watched)

    for w, g in zip(watched, gradients):
      w.assign_sub(g)

    return final_loss
  @tf.function
  def return_encoder(self):
    return self._model
    
  
  @tf.function(input_signature=[tf.TensorSpec([None], tf.dtypes.string)])  
  def validate(self,sentences):
    tokens,lookup_ids = self._tokens_to_lookup_ids(sentences)
    # Targets are the next word for each word of the sentence.
    tokens_ids_seq = lookup_ids[:, 0:-1]
    tokens_ids_target = lookup_ids[:, 1:]
    tokens_prefix = tokens[:, 0:-1]

    # Mask determining which positions we care about for a loss: all positions
    # that have a valid non-terminal token.
    mask = tf.logical_and(
        tf.logical_not(tf.equal(tokens_prefix, "")),
        tf.logical_not(tf.equal(tokens_prefix, "<E>")))

    input_mask = tf.cast(mask, tf.int32)

    lstm_output = self.model(tokens_ids_seq)
    lstm_output = tf.reshape(lstm_output, [-1,self._state_size])
    logits = self._logit_layer(lstm_output)
      

    targets = tf.reshape(tokens_ids_target, [-1])
    weights = tf.cast(tf.reshape(input_mask, [-1]), tf.float32)

    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
          labels=targets, logits=logits)

    # Final loss is the mean loss for all token losses.
    final_loss = tf.math.divide(
          tf.reduce_sum(tf.multiply(losses, weights)),
          tf.reduce_sum(weights),
          name="final_validation_loss")

    return final_loss
    
  @tf.function
  def decode_greedy(self, sequence_length, first_word):
    #initial_state = self._lstm_cell.get_initial_state(
    #    dtype=tf.float32, batch_size=1)

    sequence = [first_word]
    current_word = first_word
    current_id = tf.expand_dims(self._words_to_indices(current_word), 0)
    #current_state = initial_state

    for _ in range(sequence_length):
      #token_embeddings = tf.nn.embedding_lookup(self._embeddings, current_id)
      #token_embeddings = tf.expand_dims(token_embeddings,0)
      #logits = self.model(tf.expand_dims(token_embeddings,0))
      lstm_output = self.model(tf.expand_dims(current_id,0))
      lstm_output = tf.reshape(lstm_output, [-1,self._state_size])
      logits = self._logit_layer(lstm_output)
      softmax = tf.nn.softmax(logits)

      next_ids = tf.math.argmax(softmax, axis=1)
      next_words = self._indices_to_words(next_ids)[0]
      
      current_id = next_ids
      current_word = next_words
      sequence.append(current_word)

    return sequence


In [8]:
sentences = ["<S> hello there <E>", "<S> how are you doing today <E>","<S> I am fine thank you <E>",
             "<S> hello world <E>", "<S> who are you? <E>"]
validation_sentences = ["<S> hello there <E>", "<S> how are you doing today <E>","<S> I am fine thank you <E>"]
vocab = [
      "<S>", "<E>", "hello", "there", "how", "are", "you", "doing", "today","I","am","fine","thank","world",
    "who"
  ]

module = ULMFiTModule(vocab=vocab, emb_dim=10, buckets=1, state_size=128,n_layers=1)

for epoch in range(200):
    train_loss = module.train(tf.constant(sentences))
    validation_loss = module.validate(tf.constant(validation_sentences))
    print("Epoch ",epoch," Train loss: ",train_loss.numpy()," Validation loss ",validation_loss.numpy())




W0818 20:40:09.598569  6400 deprecation.py:323] From C:\Users\USER\Anaconda3\lib\site-packages\tensorflow\python\ops\lookup_ops.py:985: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
C:\Users\USER\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:414: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\USER\Anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:414: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch  0  Train loss:  2.7726405  Validation loss  2.7103498
Epoch  1  Train loss:  2.6993134  Validation loss  2.6590998
Epoch  2  Train loss:  2.638142  Validation loss  2.617051
Epoch  3  Train loss:  2.586891  Validation loss  2.5829546
Epoch  4  Train loss:  2.5443046  Validation loss  2.5558074
Epoch  5  Train loss:  2.5094435  Validation loss  2.534412
Epoch  6  Train loss:  2.481242  Validation loss  2.5172582
Epoch  7  Train loss:  2.4583495  Validation loss  2.5027583
Epoch  8  Train loss:  2.439287  Validation loss  2.489617
Epoch  9  Train loss:  2.4227521  Validation loss  2.4770224
Epoch  10  Train loss:  2.4078088  Validation loss  2.464573
Epoch  11  Train loss:  2.3938692  Validation loss  2.4521012
Epoch  12  Train loss:  2.3805819  Validation loss  2.4395401
Epoch  13  Train loss:  2.36773  Validation loss  2.4268622
Epoch  14  Train loss:  2.3551767  Validation loss  2.4140584
Epoch  15  Train loss:  2.3428352  Validation loss  2.401134
Epoch  16  Train loss:  2.330

Epoch  133  Train loss:  0.53576446  Validation loss  0.5000903
Epoch  134  Train loss:  0.53073055  Validation loss  0.49494675
Epoch  135  Train loss:  0.52589273  Validation loss  0.48998278
Epoch  136  Train loss:  0.5212422  Validation loss  0.48521474
Epoch  137  Train loss:  0.5167701  Validation loss  0.48062405
Epoch  138  Train loss:  0.5124686  Validation loss  0.47620535
Epoch  139  Train loss:  0.50832945  Validation loss  0.47194996
Epoch  140  Train loss:  0.50434583  Validation loss  0.4678505
Epoch  141  Train loss:  0.5005102  Validation loss  0.4638999
Epoch  142  Train loss:  0.49681613  Validation loss  0.46009117
Epoch  143  Train loss:  0.49325705  Validation loss  0.45641845
Epoch  144  Train loss:  0.48982728  Validation loss  0.4528754
Epoch  145  Train loss:  0.48652104  Validation loss  0.44945642
Epoch  146  Train loss:  0.48333263  Validation loss  0.44615623
Epoch  147  Train loss:  0.4802572  Validation loss  0.44296938
Epoch  148  Train loss:  0.4772898

In [9]:
 # We have to call this function explicitly if we want it exported, because it
  # has no input_signature in the @tf.function decorator.
decoded = module.decode_greedy(sequence_length=10, first_word=tf.constant("<S> you"))
_ = [d.numpy() for d in decoded]
print(_)


[b'<S> you', b'world', b'<E>', b'world', b'<E>', b'world', b'<E>', b'world', b'<E>', b'world', b'<E>']


In [10]:
tf.saved_model.save(module,"test")

W0818 20:41:57.326647  6400 saved_model.py:758] Skipping full serialization of Keras model <__main__.LanguageModelEncoder object at 0x000001A6A8210550>, because its inputs are not defined.


In [13]:
b = tf.saved_model.load("test")

In [14]:
d = b.decode_greedy(sequence_length=10,first_word=tf.constant("<S> Hello"))
_ = [d.numpy() for d in decoded]
print(_)

[b'<S> you', b'world', b'<E>', b'world', b'<E>', b'world', b'<E>', b'world', b'<E>', b'world', b'<E>']


# Classifier Head 


Classifier head takes in the final layer output of the languaage model and first gets the average pool and max pool of the 
final layer outputs, then passes the concatanation of last time steps hidden state, max pool results and average pool results through given number Dense-dropout-batchnormalization blocks. Finally it produces the classifier output probabilities.

In [15]:
class LanguageClassifier(Model):
    def __init__(self,language_module,num_labels,dense_units=(128,128),dropouts=(0.1,0.1)):
        
        # initialization stuff
        super(LanguageClassifier,self).__init__()
        self._language_module = language_module
        self.model_encoder = language_module.model
        
        
        # classifier head layers
        self.dense_layers = [Dense(units,activation="relu") for units in dense_units]
        self.dropout_layers = [Dropout(p) for p in dropouts]
        self.max_pool_layer = GlobalMaxPooling1D()
        self.average_pool_layer = GlobalAveragePooling1D()
        self.batchnorm_layer = BatchNormalization()
        self.n_layers = len(self.dense_layers)
        self.final_layer = Dense(num_labels,activation="sigmoid")
        
    def __call__(self,sentences):
        
        tokens,lookup_ids = self._language_module._tokens_to_lookup_ids(sentences)
        print(lookup_ids.dtype)
        self.enc_out = self.model_encoder(lookup_ids)
        last_h = self.enc_out[:,-1,:]
        max_pool_output = self.max_pool_layer(self.enc_out)
        average_pool_output = self.average_pool_layer(self.enc_out)
        
        output = concatenate([last_h,max_pool_output,average_pool_output])
        
        for i in range(self.n_layers):
            output = self.dense_layers[i](output)
            output = self.dropout_layers[i](output)
            output = self.batchnorm_layer(output)
        
        final_output = self.final_layer(output)
        return final_output        

In [16]:
model = LanguageClassifier(num_labels=2,language_module=module)

In [17]:
model.layers

In [18]:
probabilities = model(sentences)

<dtype: 'int64'>


In [19]:
probabilities

<tf.Tensor: id=30124, shape=(5, 2), dtype=float32, numpy=
array([[0.51512676, 0.5485683 ],
       [0.52851266, 0.55098623],
       [0.56506807, 0.50878805],
       [0.51537895, 0.54841405],
       [0.5163129 , 0.53004086]], dtype=float32)>

# Classifier Training 

In [20]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [21]:
labels = tf.constant([[1],[0],[1],[0],[0]])
@tf.function
def train_step(samples, labels):
  with tf.GradientTape() as tape:
    predictions = model(samples)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)
    
    
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)
    
    
EPOCHS = 5

for epoch in range(EPOCHS):
  train_step(sentences, labels)
    #test_step(validation_sentences, validation_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

<dtype: 'int64'>


W0818 20:43:08.432374  6400 util.py:244] Unresolved object in checkpoint: (root).i2w_table._initializer
W0818 20:43:08.432374  6400 util.py:244] Unresolved object in checkpoint: (root).i2w_table._initializer._filename
W0818 20:43:08.432374  6400 util.py:252] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/alpha/guide/checkpoints#loading_mechanics for details.


<dtype: 'int64'>
Epoch 1, Loss: 0.7108809351921082, Accuracy: 20.0, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 2, Loss: 0.6584046483039856, Accuracy: 60.000003814697266, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 3, Loss: 0.6259582042694092, Accuracy: 60.000003814697266, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 4, Loss: 0.5955281853675842, Accuracy: 60.000003814697266, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 5, Loss: 0.5612379908561707, Accuracy: 60.000003814697266, Test Loss: 0.0, Test Accuracy: 0.0


# Getting Predictions 

In [ ]:
predictions = (probabilities.numpy() > 0.5).astype('float32')